In [74]:
#셀레니움을 통한 드롭다운 메뉴 변경
# 출발지는 인천공항으로 모두 동일, 도착지(도시별)와 일자 변경 후 검색버튼 누르기까지

In [75]:
# 필요 패키지
import selenium
from selenium import webdriver
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from tqdm import tqdm_notebook
import time

In [76]:
# 한국 관세물류협회 물류통계-항공운임 페이지
url = 'https://www.kcla.kr/web/inc/html/4-2_1.asp'

In [77]:
driver = webdriver.Chrome('./driver/chromedriver')
driver.get(url)

C:\Users\Jun\AppData\Local\Temp\ipykernel_10188\766079133.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./driver/chromedriver')


In [78]:
cities = ['Los Angeles','New York','Chicago', 'San Francisco', 'Atlanta','Pudong','Beijing','Tianjin','Qingdao','Guangzhou','Hangzhou','Hong Kong','Kansai','Narita','Nagoya','Frankfrut','Ho Chi Minh','Penang','Kuala Lumpur','Singapore']

In [79]:
len(cities)

20

In [80]:
url = []

In [81]:
for i in tqdm_notebook(range(20)):
    # 도착지 선택
    xpath = '//*[@id="flight_port"]'
    arrive = Select(driver.find_element(By.XPATH,xpath))
    arrive.select_by_visible_text(cities[i])
    time.sleep(1)
    # 시작 연도 선택
    # 시작 월은 디폴트가 1월임
    xpath = '//*[@id="flight_year_start"]'
    startY = Select(driver.find_element(By.XPATH,xpath))
    startY.select_by_visible_text('2018')
    time.sleep(1)
    # 종료 연도 선택
    xpath = '//*[@id="flight_year_end"]'
    endY = Select(driver.find_element(By.XPATH,xpath))
    endY.select_by_visible_text('2022')
    time.sleep(1)
    # 종료 월 선택
    xpath = '//*[@id="flight_month_end"]'
    endM = Select(driver.find_element(By.XPATH,xpath))
    endM.select_by_visible_text('07')
    # 검색버튼 누르기
    xpath = '//*[@id="SubContents_Box05"]/div/div[2]/ul/li[4]/form/div/input[3]'
    submit = driver.find_element(By.XPATH,xpath)
    driver.execute_script('arguments[0].click();',submit)
    time.sleep(1)
    # 현재페이지 url 수집
    url.append(driver.current_url)


C:\Users\Jun\AppData\Local\Temp\ipykernel_10188\206347988.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(20)):


  0%|          | 0/20 [00:00<?, ?it/s]

In [82]:
len(url)

20

In [83]:

from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.request import Request
from urllib.parse import urljoin
from html_table_parser import parser_functions

import pandas as pd
import numpy as np
import ssl

In [84]:
# 테이블 만드는 함수 설치
%pip install html_table_parser

Note: you may need to restart the kernel to use updated packages.


In [85]:
# 데이터 수집 및 df생성
table = []
df = pd.DataFrame()
for i in tqdm_notebook(range(20)):
    context = ssl._create_unverified_context()
    req = Request(url[i],headers={'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'})
    html=urlopen(req,context=context)
    #파싱 후 테이블 데이터 저장
    soup = BeautifulSoup(html,'html.parser')
    data = soup.find('table')
    #table.append(parser_functions.make2d(data))
    if parser_functions.make2d(data)[1][0] != '등록된 데이터가 없습니다.':
        table.append(parser_functions.make2d(data)[1:])
    

C:\Users\Jun\AppData\Local\Temp\ipykernel_10188\2223088406.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(20)):


  0%|          | 0/20 [00:00<?, ?it/s]

In [86]:
table

[[['Los Angeles\xa0(LAX)',
   '2022.03',
   '22.00',
   '22.00',
   '21.00',
   '21.00',
   '21.00'],
  ['Los Angeles\xa0(LAX)',
   '2022.02',
   '20.00',
   '20.00',
   '19.00',
   '19.00',
   '19.00'],
  ['Los Angeles\xa0(LAX)',
   '2022.01',
   '20.00',
   '20.00',
   '19.00',
   '19.00',
   '19.00'],
  ['Los Angeles\xa0(LAX)',
   '2021.12',
   '22.000',
   '22.000',
   '21.000',
   '21.000',
   '21.000'],
  ['Los Angeles\xa0(LAX)',
   '2021.11',
   '20.000',
   '20.000',
   '19.000',
   '19.000',
   '19.000'],
  ['Los Angeles\xa0(LAX)',
   '2021.10',
   '20.000',
   '20.000',
   '19.000',
   '19.000',
   '19.000'],
  ['Los Angeles\xa0(LAX)',
   '2021.09',
   '12.000',
   '10.000',
   '9.500',
   '9.500',
   '9.500'],
  ['Los Angeles\xa0(LAX)',
   '2021.08',
   '12.000',
   '10.000',
   '9.500',
   '9.500',
   '9.500'],
  ['Los Angeles\xa0(LAX)',
   '2021.07',
   '12.000',
   '10.000',
   '9.500',
   '9.500',
   '9.500'],
  ['Los Angeles\xa0(LAX)',
   '2021.06',
   '12.00',
   '10.0

In [87]:
len(table)

20

In [88]:
fin_df = pd.DataFrame()
for i in range(0,20):
    df = pd.DataFrame(data=table[i])
    fin_df=pd.concat([fin_df,df],axis=0,ignore_index=True)

In [89]:
fin_df.rename(columns={0:'도착', 1:'년월', 2: '45kg', 3: '100kg', 4:'300kg', 5:'500kg', 6:'1톤'}, inplace=True)
fin_df

,도착,년월,45kg,100kg,300kg,500kg,1톤
0,Los Angeles (LAX),2022.03,22.00,22.00,21.00,21.00,21.00
1,Los Angeles (LAX),2022.02,20.00,20.00,19.00,19.00,19.00
2,Los Angeles (LAX),2022.01,20.00,20.00,19.00,19.00,19.00
3,Los Angeles (LAX),2021.12,22.000,22.000,21.000,21.000,21.000
4,Los Angeles (LAX),2021.11,20.000,20.000,19.000,19.000,19.000
...,...,...,...,...,...,...,...
975,Singapore (SIN),2018.05,1.84,1.51,1.51,1.51,1.51
976,Singapore (SIN),2018.04,1.79,1.46,1.46,1.46,1.46
977,Singapore (SIN),2018.03,1.79,1.46,1.46,1.46,1.46
978,Singapore (SIN),2018.02,1.79,1.46,1.46,1.46,1.46


In [90]:
df_date = fin_df['년월']

In [91]:
for i, date in enumerate(df_date):
    df_date[i] = date.replace('.','-')
df_date

0      2022-03
1      2022-02
2      2022-01
3      2021-12
4      2021-11
        ...   
975    2018-05
976    2018-04
977    2018-03
978    2018-02
979    2018-01
Name: 년월, Length: 980, dtype: object

In [92]:
fin_df['년월'] = df_date

In [93]:
fin_df

,도착,년월,45kg,100kg,300kg,500kg,1톤
0,Los Angeles (LAX),2022-03,22.00,22.00,21.00,21.00,21.00
1,Los Angeles (LAX),2022-02,20.00,20.00,19.00,19.00,19.00
2,Los Angeles (LAX),2022-01,20.00,20.00,19.00,19.00,19.00
3,Los Angeles (LAX),2021-12,22.000,22.000,21.000,21.000,21.000
4,Los Angeles (LAX),2021-11,20.000,20.000,19.000,19.000,19.000
...,...,...,...,...,...,...,...
975,Singapore (SIN),2018-05,1.84,1.51,1.51,1.51,1.51
976,Singapore (SIN),2018-04,1.79,1.46,1.46,1.46,1.46
977,Singapore (SIN),2018-03,1.79,1.46,1.46,1.46,1.46
978,Singapore (SIN),2018-02,1.79,1.46,1.46,1.46,1.46


In [94]:
fin_df.rename(columns={'도착':'공항'},inplace=True)

In [95]:
fin_df

,공항,년월,45kg,100kg,300kg,500kg,1톤
0,Los Angeles (LAX),2022-03,22.00,22.00,21.00,21.00,21.00
1,Los Angeles (LAX),2022-02,20.00,20.00,19.00,19.00,19.00
2,Los Angeles (LAX),2022-01,20.00,20.00,19.00,19.00,19.00
3,Los Angeles (LAX),2021-12,22.000,22.000,21.000,21.000,21.000
4,Los Angeles (LAX),2021-11,20.000,20.000,19.000,19.000,19.000
...,...,...,...,...,...,...,...
975,Singapore (SIN),2018-05,1.84,1.51,1.51,1.51,1.51
976,Singapore (SIN),2018-04,1.79,1.46,1.46,1.46,1.46
977,Singapore (SIN),2018-03,1.79,1.46,1.46,1.46,1.46
978,Singapore (SIN),2018-02,1.79,1.46,1.46,1.46,1.46


In [96]:
df_airport = fin_df['공항']

In [97]:
area = []
country = []
for airport in df_airport:
    if airport == 'Los Angeles (LAX)' or airport == 'New York (JFK)' or airport == 'Chicago (ORD)' or airport == 'San Francisco (SFO)' \
        or airport == 'Atlanta (ATL)':
        area.append('미주')
        country.append('미국')
    elif airport == 'Pudong (PVG)' or airport == 'BEIJING' or airport == 'Tianjin (TSN)' or airport == 'Qingdao (TAO)' \
        or airport == 'Hangzhou (HGH)' or airport == 'Beijing\xa0(PEK)' or airport  == 'Guangzhou\xa0(CAN)':
        area.append('아시아')
        country.append('중국')
    elif airport == 'Hong Kong (HKG)':
        area.append('아시아')
        country.append('홍콩')
    elif airport == 'Kansai (KIX)' or airport == 'Narita (NRT)' or airport == 'Nagoya (NGO)':
        area.append('아시아')
        country.append('일본')
    elif airport == 'Frankfrut (FRA)':
        area.append('유럽')
        country.append('독일')
    elif airport == 'Ho Chi Minh (SGN)':
        area.append('아시아')
        country.append('베트남')
    elif airport == 'Penang (PEN)' or airport == 'Kuala Lumpur (KUL)':
        area.append('아시아')
        country.append('말레이시아')
    elif airport == 'Singapore (SIN)':
        area.append('아시아')
        country.append('싱가포르')
    

In [98]:
len(area)

980

In [99]:
fin_df['지역'] = area
fin_df['국가명'] = country

In [100]:
fin_df

,공항,년월,45kg,100kg,300kg,500kg,1톤,지역,국가명
0,Los Angeles (LAX),2022-03,22.00,22.00,21.00,21.00,21.00,미주,미국
1,Los Angeles (LAX),2022-02,20.00,20.00,19.00,19.00,19.00,미주,미국
2,Los Angeles (LAX),2022-01,20.00,20.00,19.00,19.00,19.00,미주,미국
3,Los Angeles (LAX),2021-12,22.000,22.000,21.000,21.000,21.000,미주,미국
4,Los Angeles (LAX),2021-11,20.000,20.000,19.000,19.000,19.000,미주,미국
...,...,...,...,...,...,...,...,...,...
975,Singapore (SIN),2018-05,1.84,1.51,1.51,1.51,1.51,아시아,싱가포르
976,Singapore (SIN),2018-04,1.79,1.46,1.46,1.46,1.46,아시아,싱가포르
977,Singapore (SIN),2018-03,1.79,1.46,1.46,1.46,1.46,아시아,싱가포르
978,Singapore (SIN),2018-02,1.79,1.46,1.46,1.46,1.46,아시아,싱가포르


In [117]:
df_airport = list(fin_df['공항'])
df_airport

['Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles

In [118]:
for i, airport in enumerate(df_airport):
    df_airport[i] = df_airport[i].replace(' ','_')
    df_airport[i] = df_airport[i].replace('\xa0','')
    df_airport[i] = df_airport[i].replace('Beijing\xa0(PEK)','BEIJING')


In [119]:
df_airport

['Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles(LAX)',
 'Los_Angeles

In [120]:
for i, airport in enumerate(df_airport):
    df_airport[i] = df_airport[i].replace('Beijing(PEK)','BEIJING')

In [121]:
fin_df['공항'] = df_airport

In [122]:
fin_df['공항'] == 'Guangzhou(CAN)'

0      False
1      False
2      False
3      False
4      False
       ...  
975    False
976    False
977    False
978    False
979    False
Name: 공항, Length: 969, dtype: bool

In [123]:
fin_df = fin_df.drop(fin_df.loc[fin_df['공항'] == 'Guangzhou(CAN)'].index, axis=0)

In [124]:
fin_df

,공항,년월,45kg,100kg,300kg,500kg,1톤,지역,국가명
0,Los_Angeles(LAX),2022-03,22.00,22.00,21.00,21.00,21.00,미주,미국
1,Los_Angeles(LAX),2022-02,20.00,20.00,19.00,19.00,19.00,미주,미국
2,Los_Angeles(LAX),2022-01,20.00,20.00,19.00,19.00,19.00,미주,미국
3,Los_Angeles(LAX),2021-12,22.000,22.000,21.000,21.000,21.000,미주,미국
4,Los_Angeles(LAX),2021-11,20.000,20.000,19.000,19.000,19.000,미주,미국
...,...,...,...,...,...,...,...,...,...
975,Singapore(SIN),2018-05,1.84,1.51,1.51,1.51,1.51,아시아,싱가포르
976,Singapore(SIN),2018-04,1.79,1.46,1.46,1.46,1.46,아시아,싱가포르
977,Singapore(SIN),2018-03,1.79,1.46,1.46,1.46,1.46,아시아,싱가포르
978,Singapore(SIN),2018-02,1.79,1.46,1.46,1.46,1.46,아시아,싱가포르


In [125]:
fin_df.to_csv('./data/항공운임료.csv')